In [1]:
import numpy as np
# import nltk, random
# # nltk.download('punkt')
# # nltk.download('averaged_perceptron_tagger')
# from fastai.vision.widgets import *
# import ipywidgets

In [2]:
def found_poem(input_poem):
    input_poem = input_poem.replace("''","")
    input_poem = input_poem.replace("\n", "")
    tkn_poem = nltk.word_tokenize(input_poem)
    tkn_poem = nltk.pos_tag(tkn_poem)
    tkn_poem = np.array(tkn_poem)
    size = len(tkn_poem[:,0])
    start_tag = random.randint(0,int(size/2))
#     end_tag = random.randint(int(size/2),size)
    end_tag = start_tag + 300
    ordered_tags = tkn_poem[start_tag:end_tag,1]
    ordered_tags = list(filter(lambda tag: tag != ''' '' '''and tag != "'" and  tag != ''and tag != ',' and tag != '.' and tag != ';' and tag != ':' and tag != '(' and tag != ')' and tag != '$' and tag != '#' and tag != "'" and tag != "``", ordered_tags))
    unique_tags = tkn_poem[:,1]
    unique_tags = np.unique(unique_tags)
    unique_tags = unique_tags[5:len(unique_tags)]
    words = tkn_poem[:,0]
    
    # Make a dictionary with tags as keys and words for each tag as values
    words_by_tag = {}
    for i in range(len(unique_tags)):
        tag = unique_tags[i]
        tag_array = tkn_poem[tkn_poem[:,1]==tag][:,0]
        words_by_tag[tag] = tag_array

    #Convert values from array of words to 2D array of words and probability of those words appearing
    for tag in words_by_tag:  
        (unique, counts) = np.unique(words_by_tag[tag],return_counts=True)
        word_frequencies = np.array((unique,counts)).T
        word_frequencies = word_frequencies[1:np.shape(word_frequencies)[0],:]
        word_count = np.sum(word_frequencies[:,1].astype(np.float))
        word_frequencies[:,1] = word_frequencies[:,1].astype(np.float)/word_count
        words_by_tag[tag] = np.array(word_frequencies)

    ## Make array of words based on the ordered tags. 
    ## Select each word based on probability of the word for that tag
    output_poem = []
    for tag in ordered_tags:
        try:
            words = words_by_tag[tag][:,0]
            weights = words_by_tag[tag][:,1]
            if len(words) != 0:
                output_poem.append(random.choices(words,weights.astype(np.float),k=1)[0])
        except KeyError:
            continue

    output_poem = ' '.join(output_poem)
    line_break = 10
    for i in range(len(output_poem)):
        if i == line_break:
            output_poem[i]+'\n'
            line_break+=10
    return output_poem


## How to write a poem ##
## Take a poem, or two, or three ##
### Convert it to a list of words ##
### Get the probability of each word appearing ##
### Remove duplicates from the list ###
#######################################
###What I want to do ###
### Pick words based on popularity ###
### control sentence structure based on word type (verb, preposition,noun,adjective, article, etc)###
### add rhyme ###

In [3]:
## This function creates a random poem based on probability of inputs 
def probability_poem(input_poem):
    poem_np = np.array(input_poem)
    (unique, counts) = np.unique(poem_np,return_counts=True)
    word_frequencies = np.array((unique,counts)).T
    word_frequencies = word_frequencies[1:np.shape(word_frequencies)[0],:]
    word_count = np.sum(word_frequencies[:,1].astype(np.float))
    word_frequencies[:,1] = word_frequencies[:,1].astype(np.float)/word_count
    prob_of_each_word = word_frequencies[:,1].astype(np.float)
    poem = np.random.choice(word_frequencies[:,0],14*14,p=word_frequencies[:,1].astype(float))
    return poem

In [4]:
# legal_contract = open("assetPurchaseAgreement.txt",'r').read()
# emily_dickinson = open("dickinson.txt",'r').read()
diary = open("natalie.txt", 'r').read()
problems_of_philosophy = open("bertrand.txt",'r').read()
input_poem = diary + problems_of_philosophy
found_poem(input_poem);

In [6]:
def convertToListOfWords(input_poem):
    input_poem = input_poem.split(sep=' ')
    for i in range(len(input_poem)):
        #Remove all parantheticals
        s = input_poem[i].replace('(','')
        input_poem[i] = s
        s = input_poem[i].replace(')','')
        input_poem[i] = s
        #Remove linebreak
        s = input_poem[i].split(sep='\n')
        input_poem[i] = s[0]
        if len(s) > 1:
            input_poem.append(s[1])
        #Remove semicolon
        s = input_poem[i].split(sep=';')
        input_poem[i] = s[0]
        if len(s) > 1:
            input_poem.append(s[1])
    
    return input_poem

In [7]:
input_poem = ipywidgets.Textarea(placeholder="Put something here", description="Input Poem:")

In [8]:
btn_run = widgets.Button(description="Make New Poem")

In [33]:
out_pl = widgets.Output(layout={'border': '1px solid black'})

In [34]:
output_poem_lbl = widgets.Label() 

In [35]:
def on_click_poem(change):
    input_value = input_poem.value
    the_poem = found_poem(input_value)
    out_pl.clear_output()
    with out_pl: print(the_poem)    
btn_run.on_click(on_click_poem)

In [36]:
VBox([widgets.Label('Generate your poem'), input_poem, btn_run, out_pl])